In [6]:
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from datetime import datetime
import numpy as np

from dask.diagnostics import ProgressBar

import os.path


In [7]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

## Local dask

In [ ]:
from dask.distributed import Client
client = Client()   # create a local dask cluster on the local machine.
client

In [ ]:
# To close at end of session
client.close()

## Dask with gateway

In [13]:
from dask.distributed import Client
from dask_gateway import Gateway
gateway = Gateway()

In [14]:
cluster = gateway.new_cluster(worker_memory=4, worker_cores=1)
cluster.scale(4)
client = Client(cluster) # create a dask Gateway cluster
cluster

In [15]:
if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.b8d8ada755c349c08f4cb8cb36d26f24/status,


In [5]:
# To close at end of session
client.close()

2022-11-09 06:32:02,871 - distributed.worker - WARNING - Heartbeat to scheduler failed
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 223, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 1159, in heartbeat
    response = await retry_operation(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 383, in retry_operation
    return await retry(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 368, in retry
    return await coro()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/core.py", line 1154, in send_recv_from_rpc
    return await send_r

In [12]:
#closing open 
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

[]


In [6]:
# To close at end of session
client.close()

2022-11-09 06:31:07,491 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-09 06:31:07,492 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-09 06:31:07,492 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-11-09 06:31:07,533 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,537 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,561 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,571 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,667 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,671 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,695 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,719 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,800 - distributed.nanny - WARNING - Restarting worker
2022-11-09 06:31:07,804 - distributed.nanny - WARNING - Restarting worker

In [ ]:
cluster.shutdown()

## List files

In [5]:
print(s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[1])
print(s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[-2])

escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20020601-v5.4.nc
escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111004-v5.4.nc


In [6]:
print(s3.ls('escience2022/Antoine/AMSR2_ASI_sea_ice_concentration')[1])
print(s3.ls('escience2022/Antoine/AMSR2_ASI_sea_ice_concentration')[-2])

escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20120703-v5.4.nc
escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191230-v5.4.nc


## Write Monthly files : AMSR2

In [8]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[3], '%Y%m%d') for date in paths]

In [9]:
def read_daily_files(year):
    print('year '+ str(year))
    for month in range (1,13):
        sf = 'data/SIC/AMSR2_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2))
        if os.path.isfile(sf): continue  
        remote_files = 's3://escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        try:
            dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
            dset = dset.assign_coords(time=time_var)
            print('\t saving to '+ sf)
            dset.to_netcdf(sf)
            del remote_files, dset, time_var
        except OSError: 
            del time_var
            pass
        

In [10]:
[read_daily_files(year) for year in range(2012,2020)]

year 2012
year 2013
year 2014
year 2015
year 2016
year 2017
year 2018
year 2019


[None, None, None, None, None, None, None, None]

## Write Monthly files : AMSR-E

In [9]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[2], '%Y%m%d') for date in paths]

In [10]:
def read_daily_files(year):
    print('year '+ str(year))
    for month in range (1,13):
        print('month '+ str(month))
        remote_files = 's3://escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        try:
            dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
            dset = dset.assign_coords(time=time_var)
            sf = 'data/SIC/AMSR-E_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2))
            print('\t saving to '+ sf)
            dset.to_netcdf(sf)
            del remote_files, dset, time_var
        except OSError: 
            del time_var
            pass
        

In [ ]:
[read_daily_files(year) for year in range(2006,2012)]

### check

In [16]:
ds=xr.open_dataset('data/SIC/AMSR2_ASI_sea_ice_concentration_201207.nc')
ds
# ds.sel(time='20120702').z.plot()

<xarray.Dataset>
Dimensions:              (time: 30, x: 1216, y: 1792)
Coordinates:
  * x                    (x) float64 -3.847e+06 -3.841e+06 ... 3.747e+06
  * y                    (y) float64 -5.347e+06 -5.341e+06 ... 5.847e+06
  * time                 (time) datetime64[ns] 2012-07-02 ... 2012-07-31
Data variables:
    polar_stereographic  (time) |S1 ...
    z                    (time, y, x) float32 ...
Attributes:
    author:       University of Bremen, Gunnar Spreen [gunnar.spreen@uni-brem...
    Conventions:  CF-1.5
    GMT_version:  5.2.1 (r15220) [64-bit] [MP]
    history:      Mon Jan 21 18:50:27 2019: GDAL CreateCopy( /ssmi/www/htdocs...
    info:         AMSR2 sea ice concentration based on the ASI algorithm (Spr...
    title:        Produced by grdmath
    GDAL:         GDAL 2.1.3, released 2017/20/01

In [13]:
date = 'escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20120703-v5.4.nc'
datetime.strptime(date.split('/')[-1].split('-')[3], '%Y%m%d') 

datetime.datetime(2012, 7, 3, 0, 0)

In [4]:
f = 'data/SIC/AMSR2_ASI_sea_ice_concentration_201207.nc'


True